In [1]:
import os
import requests
import re
import json
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.parse
import random
import concurrent.futures
import xml.etree.ElementTree as ET
import time

total_image = 0  # Global variable to count total images downloaded

products_dict = {}
product_errors = {}
def update_json_all_file(file_path, new_data):
    # Kiểm tra xem tệp có tồn tại không
    if os.path.exists(file_path):
        # Đọc dữ liệu hiện có từ tệp
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                existing_data = json.load(file)
            except json.JSONDecodeError:
                existing_data = {}
    else:
        existing_data = {}

    # Kiểm tra nếu dữ liệu hiện có không phải là dict, biến nó thành dict
    if not isinstance(existing_data, dict):
        existing_data = {}
    for key,value in new_data.items():
        if key  in existing_data:
            existing_data[key].update(value)
        else:
            existing_data.update(new_data)    
    

    # Ghi dữ liệu đã cập nhật trở lại tệp
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

def update_json_error(file_path, new_data):
    # Kiểm tra xem tệp có tồn tại không
    if os.path.exists(file_path):
        # Đọc dữ liệu hiện có từ tệp
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                existing_data = json.load(file)
            except json.JSONDecodeError:
                existing_data = {}
    else:
        existing_data = {}

    # Kiểm tra nếu dữ liệu hiện có không phải là dict, biến nó thành dict
    if not isinstance(existing_data, dict):
        existing_data = {}

    existing_data.update(new_data)    
    

    # Ghi dữ liệu đã cập nhật trở lại tệp
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

In [2]:
file_path = "product_details.json"
with open(file_path,'r',encoding='utf-8') as file:
    try:
        products_dict = json.load(file)
    except:
        products_dict = {}
print(len(products_dict))

file_path = "product_category_error.jsonl"
with open(file_path,'r',encoding='utf-8') as file:
    try:
        products_error = json.load(file)
    except:
        products_error = {}
print(len(products_error))

0
0


In [3]:
def extract_key(url):
    parsed_url = urlparse(url)
    path_components = parsed_url.path.strip("/").split("/")
    return path_components[0]

def sanitize_file_name(text):
    text = text.replace(" ", "-")
    text = re.sub(r'[^A-Za-z0-9_-]', '', text)
    return text

def download_image(image_link, product_id, hash_product_name, view_number,param):
    global total_image  # Declare total_image as global to modify it
    
    folder = 'D:\VScode\DE\crawl_data\Beautifulsoup\Glamira\project_glamira\photo'
    file_name = f"image_{product_id}_{hash_product_name}_{param}_{view_number}"
    file_path = os.path.join(folder, file_name)
    
    resp = requests.get(image_link)
    
    if resp.status_code == 200:
        with open(f"{file_path}.jpg", 'wb') as file:
            file.write(resp.content)
        
        total_image += 1  # Increment total_image
        print(f"Total images downloaded: {total_image}")
        
        #Every 25 links image, sleep 3-5 seconds
        if total_image % 25 == 0:
            print("Sleeping for 3-5 seconds...")
            time.sleep(random.uniform(3, 5))
    
    else:
        print(f"Failed to retrieve the image {image_link}")

    return file_path

def generate_image(image_first_link, product_id, hash_product_name,param):
    parts = image_first_link.split('/view/')
    base_url_part1 = parts[0] + '/view/'
    base_url_part2 = parts[1].split('/', 1)[1]
    
    image_download_paths = []
    
    for view_number in range(1, 5):
        image_view_link = f"{base_url_part1}{view_number}/{base_url_part2}"
        image_download = download_image(image_view_link, product_id, hash_product_name, view_number,param)
        image_download_paths.append(image_download)

    return image_download_paths

def take_product_info(soup, products_dict):
    product_items = soup.find_all('li', class_=['item', 'product', 'product-item'])

    for item in product_items:
        try:
            category_label = item.find('p', class_='enable-popover popover_stone_info')['data-ajax-data']
            category_json = json.loads(category_label)
            category_id = category_json['data_filter']['id']
        except:
            category_id = "No Found Category ID"

        try:
            product_id = item.find('div', class_='price-box price-final_price')['data-product-id']
        except:
            product_id = "No Found Product ID"

        try:
            product_name = item.find('h2', class_="product-name").get_text()
            hash_product_name = sanitize_file_name(product_name)
        except:
            product_name = "No Found Product Name"

        try:
            image_link = item.find('img')['srcset'].split(',')[0].split()[0]
            image_path = generate_image(image_link, product_id, hash_product_name)
        except:
            image_path = "No found images"
            
        try:
            product_link = item.find('a', class_='product-link')['href']
            param = item.find('a', class_='product-link')['data-param']
            param_dict = json.loads(param)
            json_str = urllib.parse.urlencode(param_dict)
            product_full_link = product_link + '?' + json_str
        except:
            product_link = "No Found Product Link"

        try:
            short_description = item.find('span', class_='short-description').get_text(strip=True)
        except:
            short_description = "No Found Short Description"

        try:
            carat_info = item.find('span', class_='carat').get_text(strip=True)
        except:
            carat_info = "No Found Carat Info"

        try:
            price = item.find('span', class_='price').get_text(strip=True)
        except:
            price = "No Found Price "

        try:
            product_info = json.loads(item.find('a', class_='product-link')['data-param'])
        except:
            product_info = "No Found Details"

        try:
            image_link = item.find('img')['srcset'].split(',')[0].split()[0]
            image_path = generate_image(image_link, product_id, hash_product_name,json_str)
        except:
            image_path = "No found images"

        product_details = {
            "Product Name": product_name,
            "Product ID": product_id,
            'Product Full Link': product_full_link,
            "Category ID": category_id,
            'Price': price,
            'Carat': carat_info,
            "Detail": product_info,
            "Descript": short_description,
            'Product Image Path': image_path
        }

        if product_name not in products_dict:
            products_dict[product_name] = {short_description: product_details}
        else:
            products_dict[product_name][short_description] = product_details

def get_api_links(link_category):
    
    api_link = link_category + "?isAjax=true&sections=list_product"
    
    print(link_category)

    try:
        session = requests.Session()
        retry_strategy = requests.packages.urllib3.util.retry.Retry(
            total=3,  # Number of retries
            status_forcelist=[429, 500, 502, 503, 504],  # HTTP statuses to retry on
            allowed_methods=["HEAD", "GET", "OPTIONS"],  # Methods to retry
            backoff_factor=random.uniform(0, 10)  # Retry delay in seconds
        )
        adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)

        api_response = session.get(api_link, timeout=10)
        api_response.raise_for_status()  # Raise HTTPError for bad responses 

        if api_response.status_code == 200:
            api_content = api_response.content
            data = json.loads(api_content)
            html_content = data["list_product"]
            soup = BeautifulSoup(html_content, 'html.parser')
            take_product_info(soup, products_dict)
            last_page = int(soup.find('li', class_=['item', 'product', 'product-item'])['data-lastpage'])
            
            if last_page == 1:
                pass
            
            for i in range(2, last_page + 1):
                api_link_2 = f"{link_category}?p={i}&isAjax=true&sections=list_product"
                api_response = requests.get(api_link_2)
                api_content = api_response.content
                data = json.loads(api_content)
                html_content = data["list_product"]
                soup = BeautifulSoup(html_content, 'html.parser')
                take_product_info(soup, products_dict)

            # Save products_dict to JSON file
            output_file = 'product_details.json'
            update_json_all_file(output_file,products_dict)

            print(f"{link_category} details saved to {output_file}")
            return products_dict

        else:
            print(f"Failed to fetch data from {api_link}. Status code: {api_response.status_code}")
            output_file = 'product_category_error.jsonl'
            update_json_error(output_file,'link error')
            return {}

    except requests.RequestException as e:
        print(f"Error fetching initial data for {link_category}: {e}")
        output_file = 'product_category_error.jsonl'
        update_json_error(output_file,'link error')
        return {}


# List of XML files containing API links
xml_files = [
    'https://www.glamira.com/media/sitemap/glus/category_filter_provider-41-2.xml'
]

links = []

# Extract all https links from XML files
for xml_file in xml_files:
    response = requests.get(xml_file)
    xml_content = response.content
    root = ET.fromstring(xml_content)

    for element in root.iter():
        if element.text is not None and element.text.startswith("https:") and "carat" not in element.text:
            links.append(element.text)



# Process API links with thread method
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(get_api_links, link) for link in links[800:850]]

    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            print("Image Download Counts per link_category:")
            print(f"Total images downloaded: {total_image}")


<>:14: SyntaxWarning: invalid escape sequence '\V'
<>:14: SyntaxWarning: invalid escape sequence '\V'
C:\Users\Admin\AppData\Local\Temp\ipykernel_25652\3713987332.py:14: SyntaxWarning: invalid escape sequence '\V'
  folder = 'D:\VScode\DE\crawl_data\Beautifulsoup\Glamira\project_glamira\photo'


https://www.glamira.com/hand-of-hamsa-collection-earrings/brown-diamond/
https://www.glamira.com/hand-of-hamsa-collection-earrings/citrine/
https://www.glamira.com/hand-of-hamsa-collection-earrings/diamond/
https://www.glamira.com/hand-of-hamsa-collection-earrings/white-sapphire/
https://www.glamira.com/hand-of-hamsa-collection-earrings/swarovski-crystal/
https://www.glamira.com/hand-of-hamsa-collection-earrings/emerald/
https://www.glamira.com/hand-of-hamsa-collection-earrings/swarovski-green/
https://www.glamira.com/hand-of-hamsa-collection-earrings/fire-opal/
https://www.glamira.com/hand-of-hamsa-collection-earrings/garnet/
https://www.glamira.com/hand-of-hamsa-collection-earrings/green-amethyst/
Total images downloaded: 1
Total images downloaded: 2
Total images downloaded: 3
Total images downloaded: 4
Total images downloaded: 5
Total images downloaded: 6
Total images downloaded: 7
Total images downloaded: 8
Total images downloaded: 9
Total images downloaded: 10
Total images downloa